In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
SPY = pd.read_csv(os.path.join(os.path.abspath(os.getcwd()),'data','largecap','SPY.csv'))

In [6]:
SPY['Adj Close']

0       184.358871
1       184.260223
2       180.932541
3       179.228363
4       181.461731
           ...    
1253    317.943054
1254    317.952972
1255    319.645508
1256    319.566284
1257    317.804443
Name: Adj Close, Length: 1258, dtype: float64

In [95]:
def create_frame(target_ETF, horizon = 1):
    frame_10 = pd.read_csv(os.path.join(os.path.abspath(os.getcwd()),'data','10_ETF','SPY.csv'),usecols=['Date'])
    horizon_list = [1,2,3,5,10,20,40,60,120,250]
    if horizon not in horizon_list:
        raise ValueError("horizon must be one of [1,2,3,5,10,20,40,60,120,250]")
    
    for etf in ['SPY','IWM','EEM','TLT','LQD','TIP','IYR','GLD','OIH','FXE']:
        frame = pd.read_csv(os.path.join(os.path.abspath(os.getcwd()),'data','10_ETF',etf+'.csv'),usecols=['Volume','Adj Close'])
        frame.rename(columns={'Volume':etf+'_volume'}, inplace=True)
        if horizon == 1:
            frame['h_ret'] = (frame['Adj Close']/frame['Adj Close'].shift(1)) -1
        else:
            frame['h_ret'] = (frame['Adj Close']/frame['Adj Close'].shift(horizon)) -1
            lagged =  horizon_list[horizon_list.index(horizon)-1]
            for j in range(1,lagged+1):
                frame[etf +'_'+ str(j)+'_lag_ret'] = (frame['Adj Close'].shift(j)/frame['Adj Close'].shift(horizon+j)) -1
        
        if etf==target_ETF:
            frame['target'] = frame['Adj Close'] <= frame['Adj Close'].shift(-horizon)
        frame.drop(['Adj Close'],axis=1,inplace=True)
        frame_10 = pd.concat([frame_10, frame],axis=1) 
    return frame_10

In [96]:
test = create_frame('SPY',horizon= 2)

In [97]:
test.head(10)

Date  SPY_volume     h_ret  SPY_1_lag_ret  target  IWM_volume  \
0  2014-12-31   130333800       NaN            NaN   False    35688700   
1  2015-01-02   121465900       NaN            NaN   False    46133100   
2  2015-01-05   169632600 -0.018585            NaN    True    51141900   
3  2015-01-06   209151400 -0.027308      -0.018585    True    67446000   
4  2015-01-07   125346700  0.002925      -0.027308    True    32252100   
5  2015-01-08   147217800  0.030427       0.002925   False    28361700   
6  2015-01-09   158567300  0.009589       0.030427   False    26975200   
7  2015-01-12   144396100 -0.015784       0.009589   False    27316800   
8  2015-01-13   214553300 -0.010624      -0.015784   False    49991400   
9  2015-01-14   192991100 -0.008833      -0.010624    True    34224200   

      h_ret  IWM_1_lag_ret  EEM_volume     h_ret  ...  IYR_1_lag_ret  \
0       NaN            NaN    34940700       NaN  ...            NaN   
1       NaN            NaN    65010000       NaN  ...            NaN   
2 -0.019060            NaN    83166600 -0.030797  ...            NaN   
3 -0.030438      -0.019060    52066100 -0.021925  ...       0.015747   
4 -0.005198      -0.030438    64948700  0.017332  ...       0.010670   
5  0.029486      -0.005198    55081600  0.039030  ...       0.020371   
6  0.007196       0.029486    38550100  0.013681  ...       0.018697   
7 -0.012888       0.007196    55381800 -0.011421  ...       0.006027   
8 -0.002552      -0.012888    55565600  0.000510  ...       0.005869   
9 -0.003243      -0.002552    83761000  0.003081  ...       0.002870   

   GLD_volume     h_ret  GLD_1_lag_ret  OIH_volume     h_ret  OIH_1_lag_ret  \
0     7624200       NaN            NaN      325100       NaN            NaN   
1     7109600       NaN            NaN      303300       NaN            NaN   
2     8177400  0.019546            NaN      494500 -0.039532            NaN   
3    11238300  0.026648       0.019546      537900 -0.054783      -0.039532   
4     6434200  0.005440       0.026648      591200 -0.016522      -0.054783   
5     7033700 -0.010075       0.005440      344300  0.013533      -0.016522   
6     7962900  0.007129      -0.010075      367900 -0.001474       0.013533   
7     8318200  0.022598       0.007129      350600 -0.053991      -0.001474   
8     7693200  0.007675       0.022598      454800 -0.038371      -0.053991   
9     7727400 -0.004976       0.007675      538900 -0.001841      -0.038371   

   FXE_volume     h_ret  FXE_1_lag_ret  
0      324900       NaN            NaN  
1      497700       NaN            NaN  
2      698700 -0.013430            NaN  
3      432900 -0.008799      -0.013430  
4      692500 -0.008933      -0.008799  
5      821600 -0.009218      -0.008933  
6      569600  0.001030      -0.009218  
7      670600  0.003877       0.001030  
8      736000 -0.006260       0.003877  
9      762800 -0.004033      -0.006260  

[10 rows x 32 columns]

In [90]:
(SPY['Adj Close'] <= SPY['Adj Close'].shift(-2)).iloc[0:10]

0    False
1    False
2     True
3     True
4     True
5    False
6    False
7    False
8    False
9     True
Name: Adj Close, dtype: bool